In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
from mlxtend.frequent_patterns import apriori, association_rules

# Görev 1: Veriyi Hazırlama

In [3]:
# Adım 1: armut_data.csv dosyasını okutunuz.
armut_ = pd.read_csv('armut_data.csv')
armut = armut_.copy()
armut.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


In [4]:
# Adım 2: ServisID her bir CategoryID özelinde farklı bir hizmeti temsil etmektedir. ServiceID ve CategoryID’yi "_" ile birleştirerek bu hizmetleri
# temsil edecek yeni bir değişken oluşturunuz.
armut["Hizmet"] = [str(row[1]) + "_" + str(row[2]) for row in armut.values]
armut.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,UserId,ServiceId,CategoryId,CreateDate,Hizmet
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


In [6]:
# Adım 3: Öncelikle sadece yıl ve ay içeren yeni bir date değişkeni oluşturunuz. UserID ve yeni oluşturduğunuz date değişkenini "_" ile birleştirirek ID adında yeni bir değişkene atayınız.

armut['CreateDate'] = pd.to_datetime(armut['CreateDate'])
armut['New_Date'] = armut['CreateDate'].dt.to_period('M').astype(str)  # YYYY-MM formatında
armut['SepetID'] = armut['UserId'].astype(str) + '_' + armut['New_Date']
armut = armut[['UserId', 'ServiceId', 'CategoryId', 'CreateDate', 'Hizmet', 'New_Date', 'SepetID']]

# Çıktıyı görüntüle
print(armut.head())


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   UserId  ServiceId  CategoryId          CreateDate Hizmet New_Date  \
0   25446          4           5 2017-08-06 16:11:00    4_5  2017-08   
1   22948         48           5 2017-08-06 16:12:00   48_5  2017-08   
2   10618          0           8 2017-08-06 16:13:00    0_8  2017-08   
3    7256          9           4 2017-08-06 16:14:00    9_4  2017-08   
4   25446         48           5 2017-08-06 16:16:00   48_5  2017-08   

         SepetID  
0  25446_2017-08  
1  22948_2017-08  
2  10618_2017-08  
3   7256_2017-08  
4  25446_2017-08  


# Görev 2: Birliktelik Kuralları Üretiniz ve Öneride bulununuz

In [9]:
# Adım 1: Aşağıdaki gibi sepet, hizmet pivot table’i oluşturunuz.
pivot_table = armut.groupby(['SepetID', 'Hizmet'])['Hizmet'].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)
print(pivot_table.head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-9-9e18137f9c04>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table = armut.groupby(['SepetID', 'Hizmet'])['Hizmet'].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)


Hizmet         0_8  10_9  11_11  12_7  13_11  14_7  15_1  16_8  17_5  18_4  \
SepetID                                                                      
0_2017-08        0     0      0     0      0     0     0     0     0     0   
0_2017-09        0     0      0     0      0     0     0     0     0     0   
0_2018-01        0     0      0     0      0     0     0     0     0     0   
0_2018-04        0     0      0     0      0     1     0     0     0     0   
10000_2017-08    0     0      0     0      0     0     0     0     0     0   

Hizmet         19_6  1_4  20_5  21_5  22_0  23_10  24_10  25_0  26_7  27_7  \
SepetID                                                                      
0_2017-08         0    0     0     0     0      0      0     0     0     0   
0_2017-09         0    0     0     0     0      0      0     0     0     0   
0_2018-01         0    0     0     0     0      0      0     0     0     0   
0_2018-04         0    0     0     0     0      0      0     0 

In [10]:
# Adım 2: Birliktelik kurallarını oluşturunuz.
m = apriori(pivot_table, min_support=0.01, use_colnames=True)
n = association_rules(m, metric="support", min_threshold=0.01)
n.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(13_11),(2_0),0.056627,0.130286,0.012819,0.226382,1.737574,0.005442,1.124216,0.449965
1,(2_0),(13_11),0.130286,0.056627,0.012819,0.098394,1.737574,0.005442,1.046325,0.488074
2,(15_1),(2_0),0.120963,0.130286,0.033951,0.280673,2.154278,0.018191,1.209066,0.609539
3,(2_0),(15_1),0.130286,0.120963,0.033951,0.260588,2.154278,0.018191,1.188833,0.616073
4,(33_4),(15_1),0.027310,0.120963,0.011233,0.411311,3.400299,0.007929,1.493211,0.725728


In [11]:
# Adım 3: arl_recommender fonksiyonunu kullanarak en son 2_0 hizmetini alan bir kullanıcıya hizmet önerisinde bulununuz.
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in sorted_rules["antecedents"].items():
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))
    recommendation_list = list({item for item_list in recommendation_list for item in item_list})
    return recommendation_list[:rec_count]

arl_recommender(n, "2_0", 1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['22_0']